### Lennard-Jones evolution (LJ5 with ASE calculator)

Utilizing `ase.calculators.lj.LennardJones` an evolution of the LJ5 cluster is performed. Instead of enforcing position bounds as in the LJ example script, a simple approach is chosen:
- randomly position 5 atoms in a cell
- enlarge the cell ([100, 100, 100])
- center the atoms within the cell

In [209]:
from ase import Atoms
from clinamen2.utils.structure_setup import place_atoms_random_cube

positions = place_atoms_random_cube(
    random_seed=0, n_atoms=5, side_length=3.0
).reshape((-1, 3))

atoms = Atoms(
    "5X",
    cell=[100.0, 100.0, 100.0],
    pbc=False,
    positions=[(pos[0], pos[1], pos[2]) for pos in positions],
)
atoms.center()

founder_atoms = atoms.copy()

The founder degrees of freedom (dof) are then the flattened atom positions. One atom is fixed to reduce the dof.

In [210]:
# the founder is simply the flattened positions
def dof_from_atoms(atoms):
    """flatten the positions into an array"""

    return atoms.get_positions().flatten()[3:]


founder = dof_from_atoms(founder_atoms)

Next, a closure is instantiated that can be passed to the algorithm for loss evaluation. This needs to create an atoms object and call the energy calculation (via the calculator object).

In [211]:
import numpy as np

from ase.calculators.lj import LennardJones

from clinamen2.cmaes.params_and_state import (
    create_sample_and_sequential_evaluate,
    create_sample_from_state,
    create_update_algorithm_state,
)
from clinamen2.utils.script_functions import cma_setup


# we need a simple closure to
#  - translate between the CMA and the ASE calculator
#  - calculate the loss (LJ energy)
def get_eval_closure(founder_atoms):
    """Return a closure for transformation and evaluation"""

    calc = LennardJones()

    def evaluate_dof(dof):
        """return the LJ energy"""

        atoms = founder_atoms.copy()
        atoms.positions[1:] = dof.reshape((-1, 3))  # 1st atom fixed
        atoms.set_calculator(calc)

        energy = atoms.get_potential_energy()

        return energy

    return evaluate_dof


eval_closure = get_eval_closure(founder_atoms)

# initialize AlgorithmParameters and AlgorithmState
parameters, initial_state = cma_setup(mean=founder, step_size=0.5)

# The closures can be created by passing the AlgorithmParameters to the respective functions.
update_state = create_update_algorithm_state(parameters=parameters)
sample_individuals = create_sample_from_state(parameters)

sample_and_evaluate = create_sample_and_sequential_evaluate(
    sample_individuals=sample_individuals,
    evaluate_loss=eval_closure,
)

Perform an evolution over 1000 generations and print the resulting minimum energy.

In [212]:
from tqdm.auto import tqdm

state = initial_state
for g in tqdm(range(1000)):
    # perform one generation
    generation, state, loss = sample_and_evaluate(state=state)
    # to update the AlgorithmState pass in the sorted generation
    state = update_state(state, generation[np.argsort(loss)])

# print the minimum loss in the final generation
print(
    f"Loss {loss.min()} for individual " f"{loss.argmin()} in generation {g}."
)

  0%|          | 3/1000 [00:00<00:35, 27.98it/s]

100%|██████████| 1000/1000 [00:16<00:00, 61.46it/s]

Loss -9.04905799826517 for individual 0 in generation 999.


The result is a decent estimate of the pulished minimum for the LJ5, which is -9.103852. Visualized in a small cell.

In [213]:
result = founder_atoms.copy()
result.positions[1:] = generation[loss.argmin()].reshape((-1, 3))
result.set_cell([3, 3, 3], scale_atoms=False)
result.center()

from ase.visualize import view

view(result)

<Popen: returncode: None args: ['/home/rwanzenboeck/miniconda3/envs/c2resub/...>